Naloga pajka:
1. HTTP downloader and renderer: To retrieve and render a web page.
2. Data extractor: Minimal functionalities to extract images and hyperlinks.
3. Duplicate detector: To detect already parsed pages.
4. URL frontier: A list of URLs waiting to be parsed.
5. Datastore: To store the data and additional metadata used by the crawler.

TO-DO 2-images exctraction, duplicate detector 

In [9]:
import concurrent.futures
import threading
import psycopg2

lock = threading.Lock()

def reset_db():
    
    conn = psycopg2.connect(host="83.212.127.54",dbname='crawler', user="test", password="fricrawl")
    conn.autocommit = True
    cur = conn.cursor()
    cur.execute("DELETE FROM crawldb.image")
    cur.execute("DELETE FROM crawldb.page_data")
    cur.execute("DELETE FROM crawldb.page")
    cur.execute("DELETE FROM crawldb.site")
    cur.execute("DELETE FROM crawldb.link")


    if cur.rowcount != 0:
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()

  

def update_site_locking(domain, sitemap, robotstxt):
    
    with lock:
        try:
            conn = psycopg2.connect(host="83.212.127.54",dbname='crawler', user="test", password="fricrawl")
            conn.autocommit = True
            cur = conn.cursor()
            cur.execute("INSERT INTO crawldb.site (domain, robots_content,sitemap_content) VALUES (%s, %s, %s) RETURNING id;",
            (domain, sitemap, robotstxt))
            id = -1
            if cur.rowcount != 0:
                id = cur.fetchone()[0]

            conn.commit()
            cur.close()
            if id != -1:
                return id;
            else:
                print("Error with cur in update_site_locking!")
        except Exception as e:
            print("Error in update_site_locking: ", e)
        finally:
            if conn is not None:
                conn.close()
                    
    
                    
def update_page_locking(siteId, url,html_content, status_code, acc_time, page_type_code):
    
    with lock:
        try:
            conn = psycopg2.connect(host="83.212.127.54",dbname='crawler', user="test", password="fricrawl")
            conn.autocommit = True
            cur = conn.cursor()
            cur.execute("INSERT INTO crawldb.page (site_id, url,html_content,http_status_code,accessed_time, page_type_code) VALUES (%s,%s,%s,%s,%s,%s) RETURNING id;",
            (siteId, url, html_content,status_code,acc_time,page_type_code))
            id = -1
            if cur.rowcount != 0:
                id = cur.fetchone()[0]

            conn.commit()
            cur.close()
            if id != -1:
                return id;
            else:
                print("Error with cur in update_page_locking!")
        except Exception as e:
            print("Error in update_page_locking: ", e)
        finally:
            if conn is not None:
                conn.close()
                    
    
                    
def update_image_locking(pageId, filename, content_type, data, acc_time):
    
    with lock:
        try:
            conn = psycopg2.connect(host="83.212.127.54",dbname='crawler', user="test", password="fricrawl")
            conn.autocommit = True
            cur = conn.cursor()
            cur.execute("INSERT INTO crawldb.image (page_id,filename, content_type,data,accessed_time) VALUES (%s,%s,%s, %s, %s) RETURNING id;",
            (pageId, filename, content_type,data,acc_time))
            id = -1
            if cur.rowcount != 0:
                id = cur.fetchone()[0]

            conn.commit()
            cur.close()
            if id != -1:
                return id;
            else:
                print("Error with cur in update_image_locking!")
        except Exception as e:
            print("Error in update_image_locking: ", e)
        finally:
            if conn is not None:
                conn.close()
                    
    
                    
def update_page_data_locking(pageId, data_type_code, data):
    
    with lock:
        try:
            conn = psycopg2.connect(host="83.212.127.54",dbname='crawler', user="test", password="fricrawl")
            conn.autocommit = True
            cur = conn.cursor()
            cur.execute("INSERT INTO crawldb.page_data (page_id,data_type_code, data) VALUES (%s, %s, %s) RETURNING id;",
            (pageId, data_type_code, data))
            id = -1
            if cur.rowcount != 0:
                id = cur.fetchone()[0]

            conn.commit()
            cur.close()
            if id != -1:
                return id;
            else:
                print("Error with cur in update_page_data_locking!")
        except Exception as e:
            print("Error in update_page_data_locking: ", e)
        finally:
            if conn is not None:
                conn.close()
                    
    
                    
                                        
def update_link_locking(from_page, to_page):
    
    with lock:
        try:
            conn = psycopg2.connect(host="83.212.127.54",dbname='crawler', user="test", password="fricrawl")
            conn.autocommit = True
            cur = conn.cursor()
            cur.execute("INSERT INTO crawldb.link (from_page,to_page) VALUES (%s, %s);",
            (from_page, to_page))

            conn.commit()
            cur.close()
        except Exception as e:
            print("Error in update_link_locking: ", e)
        finally:
            if conn is not None:
                conn.close()
    
def get_domain_id_locking(domain):

    with lock:
        try:
            conn = psycopg2.connect(host="83.212.127.54",dbname='crawler', user="test", password="fricrawl")
            conn.autocommit = True
            cur = conn.cursor()
            cur.execute("SELECT id FROM crawldb.site WHERE domain = %s",
            (domain,))

            conn.commit()
            cur.close()
        except Exception as e:
            print("Error in update_link_locking: ", e)
        finally:
            if conn is not None:
                conn.close()
                
                
def get_domain_robots_locking(domain):

    with lock:
        try:
            conn = psycopg2.connect(host="83.212.127.54",dbname='crawler', user="test", password="fricrawl")
            conn.autocommit = True
            cur = conn.cursor()
            cur.execute("SELECT robots_content FROM crawldb.site WHERE id = %s",
            (domain,))
           
            conn.commit()
            cur.close()
            return id
        except Exception as e:
            print("Error in update_link_locking: ", e)
        finally:
            if conn is not None:
                conn.close()
                 
                    



In [11]:
import requests
from lxml import html
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import queue
import re
import requests
from urllib.parse import urljoin
from urllib.parse import urlparse
from PIL import Image
from url_normalize import url_normalize
import hashlib
import datetime
from threading import Thread
from time import sleep
#import magic


visited_urls = []
visited = {}
domains = {}
frontier = queue.Queue()
#WEB_DRIVER_LOCATION = "C:/Users/Pirk/Desktop/faks-mag/ekstrakcija/chromedriver.exe" #jure
#WEB_DRIVER_LOCATION = "C:/Work/Magisterij_1_leto/2.semester/ekstrakcijaSplet/Nal1/chromedriver.exe" #matjaž
WEB_DRIVER_LOCATION = "C:/Users/miham/Documents/Faks/IEPS/chromedriver.exe" #Miha
TIMEOUT = 5
#url =  "https://www.gov.si/"
sha256 = hashlib.sha256()

def is_absolute(link):
    return bool(urlparse(link).netloc)

def is_link(link):
    if (len(re.findall('.:\/\/.',link))>0):
        return True
    return False
    #logika - preveri če je stvar valid link ki ga damo v frontier
    
def make_absolute(baselink,link):
    if (True != is_absolute(link)):
        return(urljoin(baselink,link))
    elif(link[0] == '/' and link[1] == '/'):
        return 'https:' + link
    return link
    #logika - prepozna če je link relativen inga spremeni v absolutnega

def get_robotstxt(link):
    chrome_options = Options()
    chrome_options.add_argument("user-agent=fri-ieps-TEST")
    driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
    driver.get(link + '/robots.txt')
    time.sleep(TIMEOUT)
    html = driver.page_source
    bsObj = BeautifulSoup(html, 'html.parser')
    robots = bsObj.find('pre').contents
    return robots

def get_sitemap(robotstxt):
    robotarray = robotstxt.split()
    if("Sitemap:" in robotarray):
        index = robotarray.index("Sitemap:")
        return robotarray[index+1]
    return None

def getsitemapContext(url):
   site = url.find_all("sitemap")

def checkduplicate(html):
    #print(hashlib.sha224(str(html).encode("utf-8")).hexdigest())

    if (hashlib.sha224(str(html).encode("utf-8")).hexdigest() in visited):
        return visited[hashlib.sha224(str(html).encode("utf-8")).hexdigest()]
    else:
        return -1
    
def get_ending(url):
    stays_same = ['html', 'pdf', 'doc', 'docx', 'ppt', 'pptx']
    
    
    split_url = url.split('.')
    if(len(split_url)>0):
        last = split_url[-1]
        if(len(last)>4):
            return 'html'
        elif last in stays_same:
            return last
    else:
        return 'drop'

    
def isallowed(url, dissaloved):

    for link in dissaloved:
        if(link == url):
            return False
    return True
    
def crawl_site(url,siteid,dissaloved, allowed):

    WEB_PAGE_ADDRESS = url
    #print(DOMAIN)
    if url not in visited_urls:
        baseurl = re.match('^.+?[^\/:](?=[?\/]|$)', url).group(0)
        print(baseurl)

        response = requests.get(WEB_PAGE_ADDRESS)
        statusCode = response.status_code
        time.sleep(TIMEOUT)
        
        
        
        chrome_options = Options()
        chrome_options.add_argument("user-agent=fri-ieps-TEST")
        driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
        try:
            driver.get(WEB_PAGE_ADDRESS)
        except Exception as e:
            print("Error in update_link_locking: ", e)
            return
            
        acc_time = datetime.datetime.fromtimestamp(time.time())


        time.sleep(TIMEOUT)
        


        html = driver.page_source       
        isVisited = checkduplicate(html)

        if(isVisited != -1):
            pageid = update_page_locking(siteid, '','', '', acc_time, 'DUPLICATE')
            update_link_locking(isVisited, pageid)
        
        bsObj = BeautifulSoup(html, 'html.parser')
        links = bsObj.find_all('a', href=True)
        img_tags = bsObj.findAll('img')

        img_urls = [img['src'] for img in img_tags]
        img_db_objs = []    
        for img_url in img_urls:
            split_url = img_url.split('/')
            name = split_url[-1]
            filetype = name[-4:]
            name = name[:-4]
            data = img_url
            accessed_time = time.time()

            img_db = {
                'filename': name,
                'content_type': filetype,
                'data': data,
                'accessed_time': accessed_time
            }
            img_db_objs.append(img_db)

            
            
            
        
        page_type_code = ""
        #print(url_normalize(url))
        head = bsObj.find('head').contents
        if head is None:
            page_type_code = "BINARY"
        else:
            page_type_code = "HTML"
            
        
        pageid = update_page_locking(siteid, url_normalize(url),str(html), str(statusCode), acc_time, page_type_code)
        visited[hashlib.sha224(str(html).encode("utf-8")).hexdigest()] = pageid
            
            
            
        for link in links:
            if (len(link.attrs['href'])>0 and link.attrs['href'][0] != '#'):
                ending = get_ending(make_absolute(baseurl,link.attrs['href']))
                absolute_link = make_absolute(baseurl,link.attrs['href'])
                if(ending == 'html'):
                    if(isallowed(absolute_link,dissaloved)):
                        frontier.put(absolute_link)
                elif(ending == 'drop'):
                    continue
                else: 
                    #print("TU SE NEKE ZAPISE V BAZO")
                    update_page_data_locking(pageid, ending, '')
                    #zapisi v bazo
        
        for img_db in img_db_objs:
            update_image_locking(pageid,img_db['filename'], img_db['content_type'], img_db['data'], datetime.datetime.fromtimestamp(img_db['accessed_time']))
        
       
        
def pajek(url):
    
    #url = frontier.get()
    domain = urlparse(url).netloc

    siteid = '666'
    baseid = get_domain_id_locking(str(domain))
    print(baseid, "id v bazo")
    if(baseid is None):
       
        robotsstr,sitemap = robots_to_String(url)
        
        
        print((sitemap),"sitemape")
        siteid = update_site_locking(domain,robotsstr,str(sitemap))
        
        dissaloved,allowed = getAllow_Dissalow(robotsstr)

        
    else:
        robotsstr = get_domain_robots_locking(baseid)
        dissaloved,allowed = getAllow_Dissalow(robotsstr)
        siteid = baseid
        
    print(url, "from pajek")
    crawl_site(url, siteid, dissaloved, allowed)
    return



def run(max_threads):
    threads = []
    for i in range(0, max_threads):
        url = frontier.get()
        t = Thread(target=pajek, args=(url,))
        threads.append(t)
        t.start()
        
        
    
    for t in threads:
        t.join()
    
    print("DONE", len(threads))
    threads.clear()
    run(3)
    
#reset_db()
frontier.put("https://www.gov.si/") 
frontier.put("https://www.gov.si/#content")
frontier.put("https://www.gov.si/podrocja/")
run(3)

None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
https://www.gov.si/ from pajek
https://www.gov.si
https://www.gov.si/#content from pajek
https://www.gov.si
https://www.gov.si/podrocja/ from pajek
https://www.gov.si
DONE 3
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
https://www.gov.si/#content from pajek
https://www.gov.si
https://www.gov.si/#content from pajek
https://www.gov.si
https://www.gov.si/podrocja/ from pajek
https://www.gov.si
Error in update_page_locking:  invalid input syntax for integer: ""
LINE 1: ...e,accessed_time, page_type_code) VALUES (35,'','','','2022-0...
                                                             ^

Error in update_link_locking:  null value in column "to_page" violates not-null constraint

Exception in thread Thread-41:
Traceback (most recent call last):
  File "C:\Users\miham\anaconda3\envs\wier\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\miham\anaconda3\envs\wier\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-5d453df8783b>", line 221, in pajek
    crawl_site(url, siteid, dissaloved, allowed)
  File "<ipython-input-11-5d453df8783b>", line 141, in crawl_site
    img_urls = [img['src'] for img in img_tags]
  File "<ipython-input-11-5d453df8783b>", line 141, in <listcomp>
    img_urls = [img['src'] for img in img_tags]
  File "C:\Users\miham\anaconda3\envs\wier\lib\site-packages\bs4\element.py", line 1486, in __getitem__
    return self.attrs[key]
KeyError: 'src'



DONE 3
None id v bazo
['https://play.google.com/sitemaps/sitemaps-index-0.xml', 'https://play.google.com/sitemaps/sitemaps-index-1.xml', 'https://play.google.com/sitemaps/sitemaps-index-2.xml', 'https://play.google.com/sitemaps/sitemaps-index-3.xml', 'https://play.google.com/sitemaps/sitemaps-index-4.xml', 'https://play.google.com/sitemaps/sitemaps-index-5.xml', 'https://play.google.com/sitemaps/sitemaps-index-6.xml', 'https://play.google.com/sitemaps/sitemaps-index-7.xml', 'https://play.google.com/sitemaps/sitemaps-index-8.xml', 'https://play.google.com/sitemaps/sitemaps-index-9.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
https://play.google.com/store/apps/details?id=si.gov.ostanizdrav from pajek
https://play.google.com
https://www.gov.si/podrocja/ from pajek
https://www.gov.si
https://www.gov.si/drzavni-organi/ from pajek
https://www.gov.si
Error in update_page_locking:  invalid input syntax for 

Exception in thread Thread-42:
Traceback (most recent call last):
  File "C:\Users\miham\anaconda3\envs\wier\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\miham\anaconda3\envs\wier\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-11-5d453df8783b>", line 221, in pajek
    crawl_site(url, siteid, dissaloved, allowed)
  File "<ipython-input-11-5d453df8783b>", line 141, in crawl_site
    img_urls = [img['src'] for img in img_tags]
  File "<ipython-input-11-5d453df8783b>", line 141, in <listcomp>
    img_urls = [img['src'] for img in img_tags]
  File "C:\Users\miham\anaconda3\envs\wier\lib\site-packages\bs4\element.py", line 1486, in __getitem__
    return self.attrs[key]
KeyError: 'src'



DONE 3
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
https://www.gov.si/zbirke/ from pajek
https://www.gov.si
https://www.gov.si/dogodki/ from pajek
https://www.gov.si
https://www.gov.si/novice/ from pajek
https://www.gov.si
Error in update_page_locking:  duplicate key value violates unique constraint "unq_url_idx"
DETAIL:  Key (url)=(https://www.gov.si/dogodki/) already exists.

Error in update_page_locking:  invalid input syntax for integer: ""
LINE 1: ...e,accessed_time, page_type_code) VALUES (71,'','','','2022-0...
                                                             ^

Error in update_link_locking:  null value in column "to_page" violates not-null constraint
DETAIL:  Failing row contains (40, null).

Error in update_page_locking:  duplicate key value violates unique constraint "unq_url_idx"
DETAIL:  Key (url)=(https://www.gov.si/novice/) alrea

...

Exception in thread Thread-1464:
Traceback (most recent call last):
  File "C:\Users\miham\anaconda3\envs\wier\lib\site-packages\pandas\core\indexes\base.py", line 2898, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'User-agent'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\miham\anaconda3\envs\wier\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\miham\anaconda3\envs\wier\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-

https://www.gov.si/podrocja/zunanje-zadeve/#wrapper from pajek
https://www.gov.si
https://www.gov.si/podrocja/zunanje-zadeve/ from pajek
https://www.gov.si
Error in update_page_locking:  invalid input syntax for integer: ""
LINE 1: ...accessed_time, page_type_code) VALUES (1491,'','','','2022-0...
                                                             ^

Error in update_link_locking:  null value in column "to_page" violates not-null constraint
DETAIL:  Failing row contains (1429, null).

Error in update_page_locking:  invalid input syntax for integer: ""
LINE 1: ...accessed_time, page_type_code) VALUES (1492,'','','','2022-0...
                                                             ^

Error in update_link_locking:  null value in column "to_page" violates not-null constraint
DETAIL:  Failing row contains (1429, null).

Error in update_page_locking:  duplicate key value violates unique constraint "unq_url_idx"
DETAIL:  Key (url)=(https://www.gov.si/podrocja/zunanje-zadeve/) a

Error in update_page_locking:  invalid input syntax for integer: ""
LINE 1: ...accessed_time, page_type_code) VALUES (1507,'','','','2022-0...
                                                             ^

Error in update_link_locking:  null value in column "from_page" violates not-null constraint
DETAIL:  Failing row contains (null, null).

Error in update_page_locking:  duplicate key value violates unique constraint "unq_url_idx"
DETAIL:  Key (url)=(https://www.gov.si/podrocja/druzina-otroci-in-zakonska-zveza/) already exists.

Error in update_page_locking:  duplicate key value violates unique constraint "unq_url_idx"
DETAIL:  Key (url)=(https://www.gov.si/podrocja/zaposlovanje-delo-in-upokojitev/) already exists.

DONE 3
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
None id v bazo
['https://www.gov.si/sitemap.xml'] sitemape
https://www.gov.si/podrocja/kmetijstvo-gozdarstvo-in-prehrana/ from pajek
https://www.go

Error in update_page_locking:  duplicate key value violates unique constraint "unq_url_idx"
DETAIL:  Key (url)=(https://www.gov.si/podrocja/podjetnistvo-in-gospodarstvo/) already exists.

Error in update_page_locking:  duplicate key value violates unique constraint "unq_url_idx"
DETAIL:  Key (url)=(https://www.gov.si/podrocja/obramba-varnost-in-javni-red/) already exists.



KeyboardInterrupt: 

In [3]:
import pandas as pd
import os
import io
from urllib.parse import urlparse
 
ua = 'User-agent'


def get_robots_url(url):
    domain_url = '{uri.scheme}://{uri.netloc}'.format(uri=urlparse(url))
    robots_url = domain_url + '/robots.txt'
    return robots_url
 
def read_robots_txt(url):
    robot_url = get_robots_url(url)
    robot_file = os.popen(f'curl {robot_url}').read()
    return robot_file
 
def initialize_dict(url):
    robot_file = read_robots_txt(url)
    result_data_set = {ua:{}}
    for line in robot_file.split("\n"):
        if line.startswith(ua):
            result_data_set[ua].update({line.split(':')[1].strip():{}})
    keys = []
    for key in result_data_set[ua].keys():
        keys.append(key)
    return result_data_set, keys, robot_file,make_sitemaps(robot_file)

def make_sitemaps(robots):
    data = []
    lines = str(robots).splitlines()

    for line in lines:
        if line.startswith('Sitemap:'):
            split = line.split(':', maxsplit=1)
            data.append(split[1].strip())

    return data
def parse_robot(url):
    idict = initialize_dict(url)
    result_data_set = idict[0]
    keys = idict[1]
    robot_file = idict[2]
    sitemaps=idict[3]
    print_flag = False
    for i in range(len(keys)):
        if i <= len(keys)-2:
            end_str = keys[i+1]
        else:
            end_str = 'We are done'
 
        result_data_set[ua][keys[i]]['Disallow'] = []
        result_data_set[ua][keys[i]]['Allow'] = []
        for line in robot_file.split("\n"):
            if end_str in line:
                print_flag = False
            elif keys[i] in line:
                print_flag = True
            elif print_flag:
                if line.startswith('Disallow') or line.startswith('Allow'):
                    status = line.split(':')[0].strip()
                    val = line.split(':')[1].strip()
                    result_data_set[ua][keys[i]][status].append(val)
    return result_data_set,sitemaps

def robots_to_String(url):
    result_data_set,sitemaps = parse_robot(url)
    ls = {ua:[],'Status':[],'Pattern':[]}
    for k,v in result_data_set.items():
        for v in result_data_set[k]:
            for key,value in result_data_set[k][v].items():
                for value in result_data_set[k][v][key]:
                    ls[ua].append(v)
                    ls['Status'].append(key)
                    ls['Pattern'].append(make_absolute(url,value))
                    #ls['Pattern'].append(value)
    robots_df = pd.DataFrame.from_dict(ls)
    return pd.DataFrame.to_string(robots_df),sitemaps #robots_df

def getAllow_Dissalow(data): #rabi pandas DF
    data = io.StringIO(data)
    df= pd.read_csv(data, sep='\s+')
    df=df[(df["User-agent"]=="*") | (df["User-agent"]==ua)] ##vse ki niso moj uporabnik oz. *#lahko odstranim ker se me ne tičejo
    allowed=df[df['Status']=="Allow"].Pattern.tolist()
    dissaloved=df[df['Status']=="Disallow"].Pattern.tolist()
    return dissaloved,allowed

data,sitemaps=robots_to_String("https://fortune.com/robots") #klici to da dobis df
#print(robotsdf)
dissallowed,allowed=getAllow_Dissalow(data)
print(dissallowed)
print(allowed)


['https://fortune.com/wp-admin/', 'https://fortune.com/sponsored/', 'https://fortune.com/feeds/', 'https://fortune.com/feed/', 'https://fortune.com/wp-login.php', 'https://fortune.com/wp-signup.php', 'https://fortune.com/press-this.php', 'https://fortune.com/remote-login.php', 'https://fortune.com/activate/', 'https://fortune.com/cgi-bin/', 'https://fortune.com/mshots/v1/', 'https://fortune.com/next/', 'https://fortune.com/sponsored/', 'https://fortune.com/feeds/', 'https://fortune.com/feed/', 'https://fortune.com/wp-login.php', 'https://fortune.com/wp-signup.php', 'https://fortune.com/press-this.php', 'https://fortune.com/remote-login.php', 'https://fortune.com/activate/', 'https://fortune.com/cgi-bin/', 'https://fortune.com/mshots/v1/', 'https://fortune.com/next/']
['https://fortune.com/wp-admin/admin-ajax.php']


['https://fortune.com/wp-admin/', 'https://fortune.com/sponsored/', 'https://fortune.com/feeds/', 
 'https://fortune.com/feed/', 'https://fortune.com/wp-login.php', 'https://fortune.com/wp-signup.php', 
 'https://fortune.com/press-this.php', 'https://fortune.com/remote-login.php', 'https://fortune.com/activate/',
 'https://fortune.com/cgi-bin/', 'https://fortune.com/mshots/v1/', 'https://fortune.com/next/', 
 'https://fortune.com/sponsored/', 'https://fortune.com/feeds/', 'https://fortune.com/feed/', 'https://fortune.com/wp-login.php', 'https://fortune.com/wp-signup.php', 'https://fortune.com/press-this.php', 'https://fortune.com/remote-login.php', 'https://fortune.com/activate/', 'https://fortune.com/cgi-bin/', 'https://fortune.com/mshots/v1/', 'https://fortune.com/next/']
['https://fortune.com/wp-admin/admin-ajax.php']

V spodnjem delu kode so metode za pridobivanja sitemapov, potrebno je dobiti .xml naslov iz trenutnega linka - kliči get_sitemap(robots.tx), ki iz robots txt pridobi sitemap.xml, spodnje  funkcije se sprehodijo
1. poženi urlSitemap=get_site_map(robots.txt)
2. get_all_urls_siteMap(urlSitemap)

Moj naslov: WEB_DRIVER_LOCATION = "C:/Work/Magisterij_1_leto/2.semester/ekstrakcijaSplet/Nal1/chromedriver.exe"
Juretov naslov: C:/Users/Pirk/Desktop/faks-mag/ekstrakcija/chromedriver.exe

In [4]:
import pandas as pd
import urllib.request
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import xmltodict

def get_sitemaps(url): #find all sitemaps of sitemap from robot.txt
    """Scrapes an XML sitemap from the provided URL and returns XML source.
    Args:
        url (string): Fully qualified URL pointing to XML sitemap.
    Returns:
        xml (string): XML source of scraped sitemap.
    """
    response = urllib.request.urlopen(url)
    xml = BeautifulSoup(response, 
                         'lxml-xml', 
                         from_encoding=response.info().get_param('charset'))
    return xml

def get_sitemap_type(xml):
    """Parse XML source and returns the type of sitemap.
    Args:
        xml (string): Source code of XML sitemap.
    Returns:
        sitemap_type (string): Type of sitemap (sitemap, sitemapindex, or None).
    """
    sitemapindex = xml.find_all('sitemapindex')
    sitemap = xml.find_all('urlset')
    #print(sitemap)
    if sitemapindex:
        return 'sitemapindex' #vsebujejo linke na otroke
    elif sitemap:
        return 'urlset' #direktni linki
    else:
        return
    
def get_child_sitemaps(xml):
    """Return a list of child sitemaps present in a XML sitemap file.
    Args:
        xml (string): XML source of sitemap. 
    Returns:
        sitemaps (list): Python list of XML sitemap URLs.
    """
    sitemaps = xml.find_all("sitemap")
    output = []
    for sitemap in sitemaps:
        output.append(sitemap.findNext("loc").text)
  
    return output
def sitemap_to_dataframe(xml, name=None, data=None, verbose=False):
    """Read an XML sitemap into a Pandas dataframe. 

    Args:
        xml (string): XML source of sitemap. 
        name (optional): Optional name for sitemap parsed.
        verbose (boolean, optional): Set to True to monitor progress.

    Returns:
        dataframe: Pandas dataframe of XML sitemap content. 
    """

    df = pd.DataFrame(columns=['loc', 'changefreq', 'priority', 'domain', 'sitemap_name'])

    urls = xml.find_all("url")
  
    for url in urls:

        if xml.find("loc"):
            loc = url.findNext("loc").text
            parsed_uri = urlparse(loc)
            domain = '{uri.netloc}'.format(uri=parsed_uri)
        else:
            loc = ''
            domain = ''

        if xml.find("changefreq"):
            changefreq = url.findNext("changefreq").text
        else:
            changefreq = ''

        if xml.find("priority"):
            priority = url.findNext("priority").text
        else:
            priority = ''

        if name:
            sitemap_name = name
        else:
            sitemap_name = ''
              
        row = {
            'domain': domain,
            'loc': loc,
            'changefreq': changefreq,
            'priority': priority,
            'sitemap_name': sitemap_name,
        }

        if verbose:
            print(row)

        df = df.append(row, ignore_index=True)
    return df
def get_all_urls_SiteMap(url): #provide xml of a site , from robot.txt ,... 
    """Return a dataframe containing all of the URLs from a site's XML sitemaps.
    Args:
        url (string): URL of site's XML sitemap. Usually located at /sitemap.xml
    Returns:
        df (dataframe): Pandas dataframe containing all sitemap content. 

    """  
    xml = get_sitemaps(url)
    sitemap_type = get_sitemap_type(xml)
    if sitemap_type =='sitemapindex':
        sitemaps = get_child_sitemaps(xml)
    else:
        sitemaps = [url]
    df = pd.DataFrame(columns=['loc', 'changefreq', 'priority', 'domain', 'sitemap_name'])
    for sitemap in sitemaps:
        #print(sitemap)
        sitemap_xml = get_sitemaps(sitemap) # ce želimo imeti vse povezave sitemapov
        #  sitemaps_all.append(sitemap_xml) 
        df_sitemap = sitemap_to_dataframe(sitemap_xml, name=sitemap)
        print(sitemap_xml)
        #print(df_sitemap)
        #print("end")
        df = pd.concat([df, df_sitemap], ignore_index=True)
        print(sitemap)
        #file = urllib.request.urlopen(sitemap)  #odpri xml in pridobi podatke
        #data = file.read()
        #file.close()
        #data = xmltodict.parse(data)
        #print(data)
    return df

url="https://www.gov.si/sitemap.xml" #dobimo ga recimo z robots.txt
dataFrame = get_all_urls_SiteMap(url)
print(dataFrame.head())
print(dataFrame.sitemap_name.value_counts())
xml=get_sitemap(url)
#sitemap_type = get_sitemap_type(xml)
#if(sitemap_type=="sitemapindex"): #če je 
    #child_sitemaps = get_child_sitemaps(xml)
    #print(child_sitemaps)


<?xml version="1.0" encoding="utf-8"?>
<?xml-stylesheet type='text/xsl' href='https://www.gov.si/assets/sitemap/sitemapindex.xsl'?><sitemapindex xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">
<sitemap>
<loc>https://www.gov.si/assets/sitemap/de_DE/1.xml</loc>
<lastmod>2022-03-25</lastmod>
</sitemap>
<sitemap>
<loc>https://www.gov.si/assets/sitemap/en_IE/1.xml</loc>
<lastmod>2022-03-25</lastmod>
</sitemap>
<sitemap>
<loc>https://www.gov.si/assets/sitemap/es_ES/1.xml</loc>
<lastmod>2022-03-25</lastmod>
</sitemap>
<sitemap>
<loc>https://www.gov.si/assets/sitemap/hu_HU/1.xml</loc>
<lastmod>2022-03-25</lastmod>
</sitemap>
<sitemap>
<loc>https://www.gov.si/assets/sitemap/it_IT/1.xml</loc>
<lastmod>2022-03-25</lastmod>
</sitemap>
<sitemap>
<loc>https://www.gov.si/assets/sitemap/sl_SI/1.xml</loc>
<lastmod>2022-03-25</lastmod>
</sitemap>
<sitemap>
<loc>https://www.gov.si/assets/sitemap/en_IE/2.xml</loc>
<lastmod>2022-03-25</lastmod>
</sitemap>
<sitemap>
<loc>https://www.gov.si/assets/sitem